In [ ]:
import csv
import pandas as pd
from collections import defaultdict

# Helper function to determine the maximum number of columns in a CSV file.
def max_columns_in_csv(filepath):
    with open(filepath, newline='') as f:
        reader = csv.reader(f)
        return max(len(row) for row in reader)

# --- Load CSV files using the maximum number of columns ---

# total_shark_class_sub_parts.csv
max_fields_total = max_columns_in_csv('total_shark_class_sub_parts.csv')
df_total = pd.read_csv('total_shark_class_sub_parts.csv', 
                       header=None, 
                       engine='python', 
                       names=range(max_fields_total))

# recipe_book.csv
max_fields_recipe_book = max_columns_in_csv('recipe_book.csv')
df_recipe_book = pd.read_csv('recipe_book.csv', 
                             header=None, 
                             engine='python', 
                             names=range(max_fields_recipe_book))

# recipe_gathering.csv
max_fields_recipe_gathering = max_columns_in_csv('recipe_gathering.csv')
df_recipe_gathering = pd.read_csv('recipe_gathering.csv', 
                                  header=None, 
                                  engine='python', 
                                  names=range(max_fields_recipe_gathering))

# --- Build a recipe dictionary from recipe_book.csv ---
recipes = {}
for _, row in df_recipe_book.iterrows():
    product = row[0]
    ingredients = []
    # Iterate over columns in steps of two (starting at column 1)
    for i in range(1, max_fields_recipe_book, 2):
        if pd.isna(row[i]):
            break
        ingredient = row[i]
        if i+1 < max_fields_recipe_book and not pd.isna(row[i+1]):
            qty = float(row[i+1])
        else:
            qty = 0
        ingredients.append((ingredient, qty))
    recipes[product] = ingredients

# --- Build a dictionary of top-level items from total_shark_class_sub_parts.csv ---
# The first column is the product and the second column is the required quantity.
top_level = {}
for _, row in df_total.iterrows():
    product = row[0]
    qty = float(row[1])
    top_level[product] = qty

# --- Recursive function to compute base ingredients ---
requirements = defaultdict(float)

def compute_requirements(item, multiplier):
    if item in recipes:
        for ingredient, qty in recipes[item]:
            compute_requirements(ingredient, qty * multiplier)
    else:
        requirements[item] += multiplier

# Process each top-level item.
for product, qty in top_level.items():
    compute_requirements(product, qty)

df_requirements = pd.DataFrame(list(requirements.items()), 
                               columns=["Ingredient", "Total Quantity"])

# --- Process the gathering CSV ---
def combine_location(row):
    parts = [str(x) for x in row[1:] if pd.notna(x)]
    return ', '.join(parts)

df_recipe_gathering["Location Info"] = df_recipe_gathering.apply(combine_location, axis=1)
# Select only the first column (ingredient name) and the combined location info column.
df_recipe_gathering = df_recipe_gathering[[0, "Location Info"]]
df_recipe_gathering.columns = ["Ingredient", "Location Info"]

# --- Merge and output the final CSV ---
df_output = pd.merge(df_requirements, df_recipe_gathering, on="Ingredient", how="left")

# Sort the output by ingredient in alphabetical order.
df_output = df_output.sort_values("Ingredient")

df_output.to_csv('gathering_list.csv', index=False)
df_output


,Ingredient,Total Quantity,Location Info
12,Aldgoat Horn,36.0,"Eastern Thanalan ( 18.2 / 22.7 ), Dropped by M..."
21,Bomb Ash,291.0,Southern Thanalan ( 22.2 / 29.9 )
8,Cedar Log,45.0,Coerthas Western Highlands ( 31.3 / 32.1 )
6,Cobalt Ore,450.0,Northern Thanalan ( 23.1 / 24.2 )
15,Copper Ore,42.0,Central Thanalan ( 19.0 / 26.2 )
27,Darksteel Ore,63.0,"Coerthas Central Highlands ( 27.5 / 20.0 ), Ti..."
11,Electrum Ore,156.0,Upper La Noscea ( 30.4 / 25.4 )
19,Flax,36.0,South Shroud ( 17.6 / 28.6 )
14,Gold Ore,126.0,"Eastern Thanalan ( 27.3 / 21.3 ), Timed at 10:..."
26,Hardened Sap,108.0,"Nowhere ( 0.0 / 0.0 ), Sold for 200 GC Seals"


In [8]:
import csv
import pandas as pd

def max_columns_in_csv(filepath):
    with open(filepath, newline='') as f:
        reader = csv.reader(f)
        return max(len(row) for row in reader)
    
max_fields_total = max_columns_in_csv('total_shark_class_sub_parts.csv')
df_crafting_recipes = pd.read_csv('total_shark_class_sub_parts.csv', 
                                  header=None, 
                                  engine='python', 
                                  names=range(max_fields_total))

df_crafting_recipes = df_crafting_recipes.sort_values(by=0)
df_crafting_recipes.rename(columns={0: "Product", 1: "Required Quantity"}, inplace=True)
df_crafting_recipes

,Product,Required Quantity,2,3,4,5,6,7,8,9
16,Ancient Lumber,18,Petrified Log,1,Scarlet Sap,1,Wind Cluster,2.0,Ice Cluster,1.0
4,Cedar Lumber,9,Cedar Log,5,Wind Crystal,3,None,NaN,None,NaN
3,Cobalt Ingot,150,Cobalt Ore,2,Iron Ore,1,Ice/Fire Shard,5.0,None,NaN
14,Cobalt Joint Plate,54,Cobalt Ingot,1,Ice Shard,5,None,NaN,None,NaN
20,Cobalt Rivets,21,Cobalt Ingot,1,Ice/Fire Shard,5,None,NaN,None,NaN
19,Darksteel Nugget,21,Darksteel Ore,3,Ice/Fire Crystal,3,None,NaN,None,NaN
7,Electrum Ingot,39,Electrum Ore,4,Wind Shard,5,None,NaN,None,NaN
8,Horn Glue,18,Aldgoat Horn,2,Water Shard,3,None,NaN,None,NaN
2,Iron Nails,39,Iron Ingot,1,Ice/Fire Shard,1,None,NaN,None,NaN
5,Iron Rivets,39,Iron Ingot,1,Ice/Fire Shard,1,None,NaN,None,NaN


In [ ]:
import json
import requests
import pandas as pd
import re
import time

# -------------------------------
# 1. Load the item ID mapping
# -------------------------------
# Assumes a JSON file "item_ids.json" exists with a mapping structure like:
# { "12345": {"en": "Item Name", ...}, ... }
with open("item_ids.json", "r", encoding="utf-8") as f:
    item_json = json.load(f)

# Build a mapping: lower-case English item name -> item ID
item_mapping = {}
for item_id, names in item_json.items():
    en_name = names.get("en", "").strip().lower()
    if en_name:
        item_mapping[en_name] = item_id

# -------------------------------
# 2. Combine items from both lists
# -------------------------------
# For gathering items: we use the output from the gathering list cell (gathering_list.csv)
df_gathering = pd.read_csv('gathering_list.csv')[["Ingredient"]].copy()
df_gathering.rename(columns={"Ingredient": "Item Name"}, inplace=True)
df_gathering["Category"] = "Gathering"

# For crafted products: we use the output from the crafting recipes cell (df_crafting_recipes)
# (If the DataFrame isn't saved, you can re-read the CSV if you've saved it.)
df_crafting = df_crafting_recipes.copy()[["Product"]].copy()
df_crafting.rename(columns={"Product": "Item Name"}, inplace=True)
df_crafting["Category"] = "Crafting"

# Combine the two DataFrames and drop duplicates.
df_combined = pd.concat([df_gathering, df_crafting], ignore_index=True)
df_combined = df_combined.drop_duplicates(subset=["Item Name"])

# -------------------------------
# 3. Lookup item IDs using the mapping
# -------------------------------
def clean_item_name(name):
    return name.lower().strip()

def get_item_id(name):
    cleaned = clean_item_name(name)
    if cleaned in item_mapping:
        return item_mapping[cleaned]
    else:
        print(f"Error: No ID found for item '{name}' (cleaned as '{cleaned}').")
        return None

df_combined["Item ID"] = df_combined["Item Name"].apply(get_item_id)

# -------------------------------
# 4. Setup market data columns and fetch function
# -------------------------------
market_columns = [
    "minListing_world", 
    "minListing_dc", 
    "recentPurchase_world", 
    "recentPurchase_dc", 
    "averageSalePrice_dc", 
    "dailySaleVelocity_dc"
]

# Initialize these columns with None
for col in market_columns:
    df_combined[col] = None

# Default world for query (adjust if needed)
world = "Seraph"

def fetch_market_data(item_id, world):
    url = f"https://universalis.app/api/v2/aggregated/{world}/{item_id}"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if "results" in data and len(data["results"]) > 0:
                result = data["results"][0]
                # Extract required market fields
                return {
                    "minListing_world": result.get("nq", {}).get("minListing", {}).get("world", {}).get("price"),
                    "minListing_dc": result.get("nq", {}).get("minListing", {}).get("dc", {}).get("price"),
                    "recentPurchase_world": result.get("nq", {}).get("recentPurchase", {}).get("world", {}).get("price"),
                    "recentPurchase_dc": result.get("nq", {}).get("recentPurchase", {}).get("dc", {}).get("price"),
                    "averageSalePrice_dc": result.get("nq", {}).get("averageSalePrice", {}).get("dc", {}).get("price"),
                    "dailySaleVelocity_dc": result.get("nq", {}).get("dailySaleVelocity", {}).get("dc", {}).get("quantity"),
                }
            else:
                print(f"No results found for item ID {item_id}")
        else:
            print(f"Error fetching data for item ID {item_id}. Status code: {response.status_code}")
    except Exception as e:
        print(f"Exception for item ID {item_id}: {e}")
    # Return None for each field if something goes wrong.
    return {col: None for col in market_columns}

# -------------------------------
# 5. Iterate over items and fetch market data
# -------------------------------
for idx, row in df_combined.iterrows():
    item_id = row["Item ID"]
    if item_id is not None:
        market_data = fetch_market_data(item_id, world)
        for key, value in market_data.items():
            df_combined.at[idx, key] = value
        # Optional: pause between requests if needed
        time.sleep(0.5)
    else:
        print(f"Skipping market query for '{row['Item Name']}' due to missing ID.")

# -------------------------------
# 6. Save and display the augmented DataFrame
# -------------------------------
df_combined.to_csv("sub_parts_market_data.csv", index=False)
df_combined
